In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV,ParameterGrid,cross_val_score
from sklearn.naive_bayes import CategoricalNB

In [25]:
# train = pd.read_csv('data_source/train.csv')
# test = pd.read_csv('data_source/test.csv')

# train_robust = train.copy()
# test_robust = test.copy()

x_train = pd.read_csv('data_source/X_train.csv')
y_train = pd.read_csv('data_source/y_train.csv')

x_test = pd.read_csv('data_source/X_test.csv')
y_test = pd.read_csv('data_source/y_test.csv')


x_train_std = x_train.copy()
y_train_std = y_train.copy()

x_train_robust = x_train.copy()
y_train_robust = y_train.copy()

x_test_std = x_test.copy()
y_test_std = y_test.copy()

x_test_robust = x_test.copy()
y_test_robust = y_test.copy()

x_train_std = x_train_std.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_train_std = y_train_std.loc[:,'Action']
x_test_std = x_test_std.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_test_std = y_test_std.loc[:,'Action']

x_train_robust = x_train_robust.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_train_robust = y_train_robust.loc[:,'Action']
x_test_robust = x_test_robust.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_test_robust = y_test_robust.loc[:,'Action']

In [26]:
scaler = StandardScaler()
robust = RobustScaler()

x_train_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = scaler.fit_transform(x_train_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])
x_test_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = scaler.fit_transform(x_test_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])

x_train_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = robust.fit_transform(x_train_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])
x_test_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = robust.fit_transform(x_test_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])


In [11]:
clf = CategoricalNB()
clf.fit(x_train_robust,y_train_robust)
answer = clf.predict(x_test_robust)
print(classification_report(y_test_robust, answer))

              precision    recall  f1-score   support

           0       1.00      0.46      0.63     11292
           1       0.47      0.97      0.64      4496
           2       0.99      1.00      0.99      3856
           3       0.01      0.88      0.02        16

    accuracy                           0.69     19660
   macro avg       0.62      0.83      0.57     19660
weighted avg       0.88      0.69      0.70     19660



In [12]:
parameters = [
{'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
          'fit_prior': [True, False],
          'class_prior': [None, [0.1]* 4, ]
         }
]

### Parameter Grid for robust

In [13]:
best_score = 0
best_grid = ''
for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(x_train_robust,y_train_robust)
    answer = clf.predict(x_test_robust)
    f1 = f1_score(y_test_robust,answer,average='macro')

    if f1 > best_score:
        best_score = f1
        best_grid = g
print("F1: %0.5f" % best_score)
print("Grid:", best_grid)

F1: 0.61178
Grid: {'alpha': 0.01, 'class_prior': None, 'fit_prior': False}


In [14]:
clf = CategoricalNB(alpha=0.01,class_prior=None,fit_prior=False)
clf.fit(x_train_robust,y_train_robust)

train_yhat = clf.predict(x_train_robust)
train_f1 = f1_score(y_train_robust,train_yhat,average='macro')

test_yhat = clf.predict(x_test_robust)
test_f1 = f1_score(y_test_robust,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_robust,train_yhat))
print('Test Score\n',classification_report(y_test_robust, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     25830
           1       0.87      0.96      0.91     25165
           2       1.00      1.00      1.00     26245
           3       0.95      0.86      0.90     25744

    accuracy                           0.95    102984
   macro avg       0.96      0.95      0.95    102984
weighted avg       0.96      0.95      0.95    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.58      0.73     11292
           1       0.54      0.97      0.70      4496
           2       1.00      1.00      1.00      3856
           3       0.01      0.88      0.02        16

    accuracy                           0.75     19660
   macro avg       0.64      0.85      0.61     19660
weighted avg       0.89      0.75      0.77     19660

AVG F1-Score Train: 0.9537144790708485
AVG F1-Score Test: 0.6117765369343429


### Parameter Grid for Std

In [15]:
best_score = 0
best_grid = ''
for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(x_train_std,y_train_std)
    answer = clf.predict(x_test_std)
    f1 = f1_score(y_test_std,answer,average='macro')

    if f1 > best_score:
        best_score = f1
        best_grid = g
print("F1: %0.5f" % best_score)
print("Grid:", best_grid)

F1: 0.73283
Grid: {'alpha': 0.01, 'class_prior': None, 'fit_prior': True}


In [17]:
clf = CategoricalNB(alpha=0.01,class_prior=None,fit_prior=True)
clf.fit(x_train_robust,y_train_robust)

train_yhat = clf.predict(x_train_std)
train_f1 = f1_score(y_train_std,train_yhat,average='macro')

test_yhat = clf.predict(x_test_std)
test_f1 = f1_score(y_test_std,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_std,train_yhat))
print('Test Score\n',classification_report(y_test_std, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       1.00      0.82      0.90     25830
           1       0.84      0.95      0.89     25165
           2       1.00      1.00      1.00     26245
           3       0.83      0.86      0.85     25744

    accuracy                           0.91    102984
   macro avg       0.92      0.91      0.91    102984
weighted avg       0.92      0.91      0.91    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.23      0.38     11292
           1       0.39      0.97      0.56      4496
           2       1.00      1.00      1.00      3856
           3       0.01      0.88      0.01        16

    accuracy                           0.55     19660
   macro avg       0.60      0.77      0.49     19660
weighted avg       0.86      0.55      0.54     19660

AVG F1-Score Train: 0.910731665502352
AVG F1-Score Test: 0.4865793902448771


In [27]:
cross_std = cross_val_score(CategoricalNB(alpha=0.01,class_prior=None,fit_prior=True),x_train_std,y_train_std,cv=5)
print(round(np.mean(cross_std),4))

nan


/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 117, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/base.py", line 668, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/naive_bayes.py", line

In [28]:
cross_std = cross_val_score(CategoricalNB(alpha=0.01,class_prior=None,fit_prior=False),x_train_robust,y_train_robust,cv=5)
print(round(np.mean(cross_std),4))

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 117, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/base.py", line 668, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/naive_bayes.py", line

nan


/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 117, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/base.py", line 668, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
  File "/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/naive_bayes.py", line